In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model("gpt-4o-mini", model_provider="openai")

messages = [
    SystemMessage("Translate the following from English into Malayalam"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='ഹായ്!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BUSJfOqrNlyKRWKHo1YyRG4wY1Qfn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f09ae7a-99e1-4eb6-9523-ac1f635d3c29-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
for token in model.stream(messages):
    print(token.content, end="|")

|ഹ|ായ്|!||

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)


In [6]:
prompt = prompt_template.invoke({"language": "Hindi", "text": "hi!"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [7]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [8]:
response = model.invoke(prompt)
print(response.content)

नमस्ते!


In [9]:
prompt = prompt_template.invoke({"language": "Malayalam", "text": "Who are you"})

In [10]:
response = model.invoke(prompt)
response.content

'നിങ്ങൾ ആരാണ്?'

# Lang chain tool call

In [24]:
from langchain_core.tools import tool
# tools
@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]

In [14]:
model_with_tools  = model.bind_tools(tools)

In [20]:
query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(query)]
ai_message = model_with_tools.invoke(query)
messages.append(ai_message)

In [25]:
for tool_call in ai_message.tool_calls:
    selected_tool = globals()[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WSxOnUPQr0S0uABDy0ZSUXxy', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_V3F6vVtlVM7OBfV0J7kSdPP2', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 97, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BUTA1AOEe8LqEBPr5tNxcrPtmndLH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4e8ac4a6-51d7-4aa2-a896-5597d51bd442-0',

In [26]:
model_with_tools.invoke(messages)

AIMessage(content='The result of \\( 3 \\times 12 \\) is \\( 36 \\), and the result of \\( 11 + 49 \\) is \\( 60 \\).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 163, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BUTGeFM4PXpeUnPh5PKJ3tiFCcRzW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ac500205-0ddf-44be-a15f-465153b17f2c-0', usage_metadata={'input_tokens': 163, 'output_tokens': 38, 'total_tokens': 201, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Stream Tool call

In [28]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [29]:
query = "What is 3 * 12? Also, what is 11 + 49?"
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_p4l1nmf2bYZa0Xa8pBzMRYAu', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'add', 'args': '', 'id': 'call_oMNPlRjo2KaXBG9hmtxvo37n', 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[]
